In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from delta import *

In [2]:
builder = pyspark.sql.SparkSession.builder.appName("DeltaApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 

In [3]:
spark = configure_spark_with_delta_pip(builder).master('local[*]').getOrCreate()

In [4]:
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://minio:9000")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", "FFD34B2AC56E76E8BB9E7EFD7D283")
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "47A13EFAB1CA1F253FD6F56DEF769")
spark._jsc.hadoopConfiguration().set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
spark._jsc.hadoopConfiguration().set("spark.jars", "postgresql-42.2.14.jar")

### 1. Leer Parquet

In [9]:
df = spark.read.parquet("Resultados.parquet")

In [10]:
df.show()

+--------------------+------+
|             country|Numero|
+--------------------+------+
|                    |    15|
|       United States|  1080|
|        South Africa|    91|
|      United Kingdom|   407|
|              Russia|   230|
|            Thailand|    48|
|              Canada|   318|
|           Australia|    93|
|           Singapore|    11|
|             Belgium|    27|
|              Mexico|   439|
|               Spain|   166|
|              France|   119|
|United Arab Emirates|    31|
|   Republic of Korea|     8|
|            Pakistan|    29|
|               Libya|    26|
|              Gambia|     6|
|         Ivory Coast|    17|
|             Ukraine|    89|
+--------------------+------+
only showing top 20 rows



### 2. Guardar el dataframe en Minio

In [17]:
df.write.format('json').options(delimiter='|').mode('overwrite').save('s3a://landing/BTC')